In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import time

In [ ]:
orders = pd.read_csv('order_brush_order.csv.zip', compression='zip', 
                                 header=0, sep=',', quotechar='"')

In [ ]:
orders.shape

In [ ]:
orders.head(20)

In [ ]:
orders['event_time'] = pd.to_datetime(orders['event_time'])  

In [ ]:
orders['shopid'].value_counts()

In [ ]:
group_shops = orders.groupby('shopid')

In [ ]:
group_shops.groups

In [ ]:
group_shops.groups.keys()

In [ ]:
for (shopid, group) in orders.groupby('shopid'):
    print(group.head(5))

In [ ]:
for (shopid, group) in orders.groupby('shopid'):
    print(group.head(5))

In [ ]:
shopid_10159 = group_shops.get_group(181009364)
shopid_10159

In [ ]:
shopid_10159['timestamp'] = shopid_10159["event_time"].values.astype(np.int64) // 10 ** 9
shopid_10159

In [ ]:
shopid_10159.sort_values(by=['timestamp'], inplace=True)
shopid_10159 = shopid_10159.reset_index(drop = True)
shopid_10159

In [ ]:
# df_order_proportion = pd.DataFrame(columns = ["index","order_proportion"]) #建立一個空的 dataframe

# # 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
# for i in range(shopid_10159["timestamp"].size):
#     flag = shopid_10159.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
#     print("flag: " + str(flag))
#     mask = (shopid_10159["timestamp"] >= flag) & (shopid_10159["timestamp"] <= (flag+3600))
#     df_small = shopid_10159.loc[mask]
#     print(df_small)
    
    
#     number_of_orders = df_small.shape[0]
#     print("number_of_orders: " + str(number_of_orders))
#     number_of_unique_buyers = df_small.userid.nunique()
#     print("number_of_unique_buyers: " + str(number_of_unique_buyers))
#     print("\n")
#     concentrate_rate = number_of_orders/number_of_unique_buyers
        
#     if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
#         small_order_proportion = (df_small["userid"].value_counts()/df_small.shape[0]).to_frame(name = "order_proportion")
#         small_order_proportion.reset_index(level=0, inplace=True)
#         df_order_proportion = pd.concat([df_order_proportion, small_order_proportion], axis=0)

In [ ]:
df_order_proportion

In [ ]:
df_result = pd.DataFrame(columns = ["shopid","userid"]) # 這行code要放外層迴圈外面！！！

df_order_brushing = pd.DataFrame(columns = ["orderid","shopid","userid","event_time","timestamp"]) #建立一個空的 dataframe

# 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
for i in range(shopid_10159["timestamp"].size):
    flag = shopid_10159.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
    print("flag: " + str(flag))
    mask = (shopid_10159["timestamp"] >= flag) & (shopid_10159["timestamp"] <= (flag+3600))
    df_small = shopid_10159.loc[mask]
    print(df_small)
    
    
    number_of_orders = df_small.shape[0]
    print("number_of_orders: " + str(number_of_orders))
    number_of_unique_buyers = df_small.userid.nunique()
    print("number_of_unique_buyers: " + str(number_of_unique_buyers))
    print("\n")
    concentrate_rate = number_of_orders/number_of_unique_buyers
        
    if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
        df_order_brushing = pd.concat([df_order_brushing, df_small], axis=0)

        
# calculate the concentrate rate
total_brushing = df_order_brushing.shape[0] # total number of orders during all order brushing periods
df_order_proportion = (df_order_brushing["userid"].value_counts()/total_brushing).to_frame(name = "order_proportion")
df_order_proportion.reset_index(level=0, inplace=True)
df_order_proportion.columns =['userid', 'order_proportion']

# find out which user is the suspicious buyer (having higest order proportion)
max_value = df_order_proportion['order_proportion'].max()
#use a list to save the suspicious buyers
max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
max_order_brushing_list.sort() # 讓這些 userid 照大小排好

# concat 到 df_output 上面
if(len(max_order_brushing_list)==0):
    df_result = df.append({'shopid': shopid, 'userid':0}, ignore_index=True)

else if(len(max_order_brushing_list)==1):
    userid_result = max_order_brushing_list[0]
    df_result = df.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)

else:
    userid_result = '&'
    userid_result = userid_result.join(map(str, max_order_brushing_list)) #先轉換成 string 後，再用&把每個元素連在一起
    df_result = df.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)



In [ ]:
df_order_brushing

In [ ]:
df_order_proportion

In [ ]:
max_value = df_order_proportion['order_proportion'].max()
#用一個list把這些洗評分的使用者存起來
max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
max_order_brushing_list.sort() # 讓這些 userid 照大小排好
print(max_order_brushing_list)

In [ ]:
userid_result = '&'
userid_result = userid_result.join(map(str, max_order_brushing_list))
print(userid_result)

In [ ]:
df_order_proportion = (shopid_10159["userid"].value_counts()/shopid_10159.shape[0]).to_frame(name = "order_proportion")
df_order_proportion.reset_index(level=0, inplace=True)
df_order_proportion

## 來把全部組合在一起

In [ ]:
df_result = pd.DataFrame(columns = ["shopid","userid"]) # 這行code要放外層迴圈外面！！！

for (shopid, group) in orders.groupby('shopid'):

    df_order_brushing = pd.DataFrame(columns = ["orderid","shopid","userid","event_time","timestamp"]) #建立一個空的 dataframe

    group['timestamp'] = group["event_time"].values.astype(np.int64) // 10 ** 9
    group.sort_values(by=['timestamp'], inplace=True)
    group = group.reset_index(drop = True)

    # 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
    for i in range(group["timestamp"].size):
        flag = group.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
#         print("flag: " + str(flag))
        mask = (group["timestamp"] >= flag) & (group["timestamp"] <= (flag+3600))
        df_small = group.loc[mask]
#         print(df_small)


        number_of_orders = df_small.shape[0]
#         print("number_of_orders: " + str(number_of_orders))
        number_of_unique_buyers = df_small.userid.nunique()
#         print("number_of_unique_buyers: " + str(number_of_unique_buyers))
#         print("\n")
        concentrate_rate = number_of_orders/number_of_unique_buyers

        if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
            df_order_brushing = pd.concat([df_order_brushing, df_small], axis=0)


    # calculate the concentrate rate
    total_brushing = df_order_brushing.shape[0] # total number of orders during all order brushing periods
    df_order_proportion = (df_order_brushing["userid"].value_counts()/total_brushing).to_frame(name = "order_proportion")
    df_order_proportion.reset_index(level=0, inplace=True)
    df_order_proportion.columns =['userid', 'order_proportion']

    # find out which user is the suspicious buyer (having higest order proportion)
    max_value = df_order_proportion['order_proportion'].max()
    #use a list to save the suspicious buyers
    max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
    max_order_brushing_list.sort() # 讓這些 userid 照大小排好

    # concat 到 df_output 上面
    if(len(max_order_brushing_list)==0):
        df_result = df_result.append({'shopid': shopid, 'userid':0}, ignore_index=True)

    elif(len(max_order_brushing_list)==1):
        userid_result = max_order_brushing_list[0]
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)

    else:
        userid_result = '&'
        userid_result = userid_result.join(map(str, max_order_brushing_list)) #先轉換成 string 後，再用&把每個元素連在一起
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)



In [ ]:
import os
print("The file will be saved under this path: ", os.getcwd())
export_csv = test_df.to_csv (r'submission_'+time.strftime("%Y-%m-%d %H%M%S", time.localtime())+'.csv', 
                             index = None, header = True) 